# Estimation of bootstrap covariances for galaxy-forest cross-correlation
We randomly resample (with replacement) both the galaxy positios and Ly-a forest pixels.

*** this version runs one of them a few times to test the covariance ***

### Read in galaxy positions and forest pixels.

In [1]:
import numpy as np
import copy
import time as time
import lyafxcorr_kg as xcorr

# Set up matplotlib and use a nicer set of plot parameters
%config InlineBackend.rc = {}
import matplotlib as mpl
mpl.rc('mathtext',fontset='stixsans')
mpl.rc('figure', facecolor="white")
#matplotlib.rc_file("../../templates/matplotlibrc")
import matplotlib.pyplot as plt
#import matplotlib.colors as colors
%matplotlib inline

from astropy.cosmology import FlatLambdaCDM
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord


# Define cosmology
cosmo = FlatLambdaCDM(H0=70, Om0=0.31)
    
# Read forest pixels    
LyaPix = xcorr.lyapix("pixel_radecz.bin",cosmo=cosmo)

print("Read in %i Ly-a forest pixels" % LyaPix.npix)
npix = LyaPix.npix

# Done with forest pixels. Now read galaxy positions
galfil = '/Users/kheegan/lya/3d_recon/map2017/crosscorr/cat_galxcorr_cl2017.dat'
gal = ascii.read(galfil)
#print(gal.columns)

gal_3d = gal[gal['source'] == '3DHST']
gal_zD = gal[gal['source'] == 'zDEEP']
gal_mosdef = gal[gal['source'] == 'MOSDEF']
gal_vuds   = gal[gal['source']=='VUDS']
gal_clamato=gal[gal['source']=='CLAMATO']

print('Read in %i 3D-HST galaxies, %i zCOSMOS-Deep galaxies, %i MOSDEF galaxies,' 
      '%i VUDS galaxies, %i CLAMATO galaxies' % 
      (len(gal_3d), len(gal_zD), len(gal_mosdef), len(gal_vuds), len(gal_clamato)) )

# Convert to 3D Sky positions
Coord_3d     = SkyCoord(ra=gal_3d['ra'] * u.degree, dec=gal_3d['dec'] * u.degree,
                  distance=cosmo.comoving_distance(gal_3d['zspec']))
Coord_zD     = SkyCoord(ra=gal_zD['ra'] * u.degree, dec=gal_zD['dec'] * u.degree,
                   distance=cosmo.comoving_distance(gal_zD['zspec']))
Coord_vuds     = SkyCoord(ra=gal_vuds['ra'] * u.degree, dec=gal_vuds['dec'] * u.degree,
                   distance=cosmo.comoving_distance(gal_vuds['zspec']))
Coord_mosdef = SkyCoord(ra=gal_mosdef['ra'] * u.degree, dec=gal_mosdef['dec'] * u.degree,
                   distance=cosmo.comoving_distance(gal_mosdef['zspec']))
Coord_clamato = SkyCoord(ra=gal_clamato['ra'] * u.degree, dec=gal_clamato['dec'] * u.degree,
                   distance=cosmo.comoving_distance(gal_clamato['zspec']))

# Read in bin edges
PiBin_fil = 'bins16_pi_0-40hMpc.txt'
SigBin_fil = 'bins10_sigma_0-30hMpc.txt'

PiBins0 = ascii.read(PiBin_fil)
SigBins0 = ascii.read(SigBin_fil)

PiEdges = PiBins0['pi_edges'].data
SigEdges = SigBins0['sigma_edges'].data

# Convert bin boundaries from Mpc/h to Mpc
PiEdges  = PiEdges/(len(PiEdges)*[cosmo.h])
SigEdges = SigEdges/(len(SigEdges)*[cosmo.h])
PiBound = (min(PiEdges), max(PiEdges) )

Read in 89974 Ly-a forest pixels
Read in 226 3D-HST galaxies, 491 zCOSMOS-Deep galaxies, 121 MOSDEF galaxies,428 VUDS galaxies, 101 CLAMATO galaxies


Compute the covariance matrix. We compare the diagonal terms with the variance as a sanity check. 

### Bootstrap VUDS Sample

In [2]:
nsamp=200
np.random.seed(seed=1223456)

GalCoords = Coord_vuds
ngal = len(GalCoords)

# Initialize output array to store all the bootstrap samples
XCorrSamples_vuds = np.empty([len(SigEdges)-1, len(PiEdges)-1, nsamp])

# This is the loop over the desired resamples
for ii in range(0,nsamp):
    if (ii % 5) == 0:
        print("Iteration #", ii)
    # Make a copy of the pixels and resample
    LyaPixTmp = copy.deepcopy(LyaPix)
    LyaPixTmp = LyaPixTmp.resample()
    # Resample galaxy positions
    GalCoordTmp = GalCoords[np.random.choice(ngal,ngal,replace=True)]
    XCorrTmp, _ = xcorr.xcorr_gal_lya(GalCoordTmp, LyaPixTmp,SigEdges, PiEdges,
                                      cosmo=cosmo,verbose=0)
    XCorrSamples_vuds[:,:,ii] = XCorrTmp

# First, reshape the bootstrap samples so that it has dimensions (nbin, nsamp)
XBootReshaped = XCorrSamples_vuds.reshape(-1, XCorrSamples_vuds.shape[-1])
Covar_vuds = np.cov(XBootReshaped)

np.save("covar_vuds_n1.npy", Covar_vuds)
np.save("bootsamp_vuds_n1.npy", XCorrSamples_vuds)

Covar1 = Covar_vuds

Iteration # 0
Iteration # 5
Iteration # 10
Iteration # 15
Iteration # 20
Iteration # 25
Iteration # 30
Iteration # 35
Iteration # 40
Iteration # 45
Iteration # 50
Iteration # 55
Iteration # 60
Iteration # 65
Iteration # 70
Iteration # 75
Iteration # 80
Iteration # 85
Iteration # 90
Iteration # 95
Iteration # 100
Iteration # 105
Iteration # 110
Iteration # 115
Iteration # 120
Iteration # 125
Iteration # 130
Iteration # 135
Iteration # 140
Iteration # 145
Iteration # 150
Iteration # 155
Iteration # 160
Iteration # 165
Iteration # 170
Iteration # 175
Iteration # 180
Iteration # 185
Iteration # 190
Iteration # 195


In [3]:
# Initialize output array to store all the bootstrap samples
XCorrSamples_vuds = np.empty([len(SigEdges)-1, len(PiEdges)-1, nsamp])

# This is the loop over the desired resamples
for ii in range(0,nsamp):
    if (ii % 5) == 0:
        print("Iteration #", ii)
    # Make a copy of the pixels and resample
    LyaPixTmp = copy.deepcopy(LyaPix)
    LyaPixTmp = LyaPixTmp.resample()
    # Resample galaxy positions
    GalCoordTmp = GalCoords[np.random.choice(ngal,ngal,replace=True)]
    XCorrTmp, _ = xcorr.xcorr_gal_lya(GalCoordTmp, LyaPixTmp,SigEdges, PiEdges,
                                      cosmo=cosmo,verbose=0)
    XCorrSamples_vuds[:,:,ii] = XCorrTmp

# First, reshape the bootstrap samples so that it has dimensions (nbin, nsamp)
XBootReshaped = XCorrSamples_vuds.reshape(-1, XCorrSamples_vuds.shape[-1])
Covar_vuds = np.cov(XBootReshaped)

np.save("covar_vuds_n2.npy", Covar_vuds)
np.save("bootsamp_vuds_n2.npy", XCorrSamples_vuds)

Covar2 = Covar_vuds

Iteration # 0
Iteration # 5
Iteration # 10
Iteration # 15
Iteration # 20
Iteration # 25
Iteration # 30
Iteration # 35
Iteration # 40
Iteration # 45
Iteration # 50
Iteration # 55
Iteration # 60
Iteration # 65
Iteration # 70
Iteration # 75
Iteration # 80
Iteration # 85
Iteration # 90
Iteration # 95
Iteration # 100
Iteration # 105
Iteration # 110
Iteration # 115
Iteration # 120
Iteration # 125
Iteration # 130
Iteration # 135
Iteration # 140
Iteration # 145
Iteration # 150
Iteration # 155
Iteration # 160
Iteration # 165
Iteration # 170
Iteration # 175
Iteration # 180
Iteration # 185
Iteration # 190
Iteration # 195


In [4]:
# Initialize output array to store all the bootstrap samples
XCorrSamples_vuds = np.empty([len(SigEdges)-1, len(PiEdges)-1, nsamp])

# This is the loop over the desired resamples
for ii in range(0,nsamp):
    if (ii % 5) == 0:
        print("Iteration #", ii)
    # Make a copy of the pixels and resample
    LyaPixTmp = copy.deepcopy(LyaPix)
    LyaPixTmp = LyaPixTmp.resample()
    # Resample galaxy positions
    GalCoordTmp = GalCoords[np.random.choice(ngal,ngal,replace=True)]
    XCorrTmp, _ = xcorr.xcorr_gal_lya(GalCoordTmp, LyaPixTmp,SigEdges, PiEdges,
                                      cosmo=cosmo,verbose=0)
    XCorrSamples_vuds[:,:,ii] = XCorrTmp

# First, reshape the bootstrap samples so that it has dimensions (nbin, nsamp)
XBootReshaped = XCorrSamples_vuds.reshape(-1, XCorrSamples_vuds.shape[-1])
Covar_vuds = np.cov(XBootReshaped)

np.save("covar_vuds_n3.npy", Covar_vuds)
np.save("bootsamp_vuds_n3.npy", XCorrSamples_vuds)

Covar3 = Covar_vuds

Iteration # 0
Iteration # 5
Iteration # 10
Iteration # 15
Iteration # 20
Iteration # 25
Iteration # 30
Iteration # 35
Iteration # 40
Iteration # 45
Iteration # 50
Iteration # 55
Iteration # 60
Iteration # 65
Iteration # 70
Iteration # 75
Iteration # 80
Iteration # 85
Iteration # 90
Iteration # 95
Iteration # 100
Iteration # 105
Iteration # 110
Iteration # 115
Iteration # 120
Iteration # 125
Iteration # 130
Iteration # 135
Iteration # 140
Iteration # 145
Iteration # 150
Iteration # 155
Iteration # 160
Iteration # 165
Iteration # 170
Iteration # 175
Iteration # 180
Iteration # 185
Iteration # 190
Iteration # 195


In [5]:
# Initialize output array to store all the bootstrap samples
XCorrSamples_vuds = np.empty([len(SigEdges)-1, len(PiEdges)-1, nsamp])

# This is the loop over the desired resamples
for ii in range(0,nsamp):
    if (ii % 5) == 0:
        print("Iteration #", ii)
    # Make a copy of the pixels and resample
    LyaPixTmp = copy.deepcopy(LyaPix)
    LyaPixTmp = LyaPixTmp.resample()
    # Resample galaxy positions
    GalCoordTmp = GalCoords[np.random.choice(ngal,ngal,replace=True)]
    XCorrTmp, _ = xcorr.xcorr_gal_lya(GalCoordTmp, LyaPixTmp,SigEdges, PiEdges,
                                      cosmo=cosmo,verbose=0)
    XCorrSamples_vuds[:,:,ii] = XCorrTmp

# First, reshape the bootstrap samples so that it has dimensions (nbin, nsamp)
XBootReshaped = XCorrSamples_vuds.reshape(-1, XCorrSamples_vuds.shape[-1])
Covar_vuds = np.cov(XBootReshaped)

np.save("covar_vuds_n4.npy", Covar_vuds)
np.save("bootsamp_vuds_n4.npy", XCorrSamples_vuds)

Covar4 = Covar_vuds

Iteration # 0
Iteration # 5
Iteration # 10
Iteration # 15
Iteration # 20
Iteration # 25
Iteration # 30
Iteration # 35
Iteration # 40
Iteration # 45
Iteration # 50
Iteration # 55
Iteration # 60
Iteration # 65
Iteration # 70
Iteration # 75
Iteration # 80
Iteration # 85
Iteration # 90
Iteration # 95
Iteration # 100
Iteration # 105
Iteration # 110
Iteration # 115
Iteration # 120
Iteration # 125
Iteration # 130
Iteration # 135
Iteration # 140
Iteration # 145
Iteration # 150
Iteration # 155
Iteration # 160
Iteration # 165
Iteration # 170
Iteration # 175
Iteration # 180
Iteration # 185
Iteration # 190
Iteration # 195


In [10]:
XCorr_vuds = np.load("xcorr_vuds_v0.npy")
XNull_vuds  = np.load("xcorr_rand_vuds_v0.npy")
DeltaVec = XCorr_vuds.reshape(XCorr_vuds.size) - XNull_vuds.reshape(XNull_vuds.size)

# Read in bin edges
PiBin_fil = 'bins16_pi_0-40hMpc.txt'
SigBin_fil = 'bins10_sigma_0-30hMpc.txt'

PiBins0 = ascii.read(PiBin_fil)
SigBins0 = ascii.read(SigBin_fil)

PiEdges = PiBins0['pi_edges'].data
SigEdges = SigBins0['sigma_edges'].data

# Convert bin boundaries from Mpc/h to Mpc
PiEdges  = PiEdges/(len(PiEdges)*[cosmo.h])
SigEdges = SigEdges/(len(SigEdges)*[cosmo.h])

# Calculate significance w.r.t. null with the different covariances

# Round 1
Cov = Covar1
InvCov = np.linalg.pinv(Cov)

minusLogL = 0.5 * np.dot(DeltaVec.transpose(), np.dot(InvCov, DeltaVec))

print("Detection significance = %f sigma" % np.sqrt(minusLogL))
signif1 = np.sqrt(minusLogL)

#Round 2
Cov = Covar2
InvCov = np.linalg.pinv(Cov)

minusLogL = 0.5 * np.dot(DeltaVec.transpose(), np.dot(InvCov, DeltaVec))

print("Detection significance = %f sigma" % np.sqrt(minusLogL))
signif2 = np.sqrt(minusLogL)

# Round 3
Cov = Covar3
InvCov = np.linalg.pinv(Cov)

minusLogL = 0.5 * np.dot(DeltaVec.transpose(), np.dot(InvCov, DeltaVec))

print("Detection significance = %f sigma" % np.sqrt(minusLogL))
signif3 = np.sqrt(minusLogL)

#Round 4
Cov = Covar4
InvCov = np.linalg.pinv(Cov)

minusLogL = 0.5 * np.dot(DeltaVec.transpose(), np.dot(InvCov, DeltaVec))

print("Detection significance = %f sigma" % np.sqrt(minusLogL))
signif4 = np.sqrt(minusLogL)

Detection significance = 14.553801 sigma
Detection significance = 12.704766 sigma
Detection significance = 14.582218 sigma
Detection significance = 14.571788 sigma
